In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras import applications
from keras import optimizers
from keras.layers.normalization import BatchNormalization

train_data_dir = 'dataset/train1'
validation_data_dir = 'dataset/validation1'

Using TensorFlow backend.


In [0]:
nb_train_samples = 1000
nb_validation_samples = 200
epochs = 30
batch_size = 20
img_width, img_height = 224, 224

top_model_weights_path = 'bottleneck_fc_model.h5'

model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')

top_model = Sequential()

top_model.add(Flatten(input_shape=model.output_shape[1:]))

top_model.add(Dense(256))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.25))
top_model.add(Dense(1))
top_model.add(Activation('sigmoid'))

top_model.load_weights(top_model_weights_path)

model = Model(input= model.input, output= top_model(model.output))

for layer in model.layers[:15]:
    layer.trainable = False
    
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

W0710 16:03:16.124215 140702301972352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0710 16:03:16.173439 140702301972352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0710 16:03:16.185482 140702301972352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0710 16:03:16.223176 140702301972352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



58892288/58889256 [==============================] - 1s 0us/step


W0710 16:03:17.826247 140702301972352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0710 16:03:17.827583 140702301972352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0710 16:03:21.126985 140702301972352 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model loaded.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
W0710 16:03:21.949894 140702301972352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0710 16:03:21.960000 140702301972352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


The saved weights from the previous experiment ```transfer-learning-bottleneck.ipynb``` are used for finetuning the model.





In [0]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=30, validation_data=<keras_pre..., steps_per_epoch=50, validation_steps=200)`


Epoch 1/30
50/50 [==============================] - 447s 9s/step - loss: 0.4058 - acc: 0.8410 - val_loss: 0.5539 - val_acc: 0.7750
Epoch 2/30
50/50 [==============================] - 85s 2s/step - loss: 0.4161 - acc: 0.8230 - val_loss: 0.4591 - val_acc: 0.7850
Epoch 3/30
50/50 [==============================] - 84s 2s/step - loss: 0.3459 - acc: 0.8460 - val_loss: 0.4518 - val_acc: 0.8100
Epoch 4/30
50/50 [==============================] - 84s 2s/step - loss: 0.3263 - acc: 0.8690 - val_loss: 0.4832 - val_acc: 0.8150
Epoch 5/30
50/50 [==============================] - 84s 2s/step - loss: 0.3310 - acc: 0.8600 - val_loss: 0.4099 - val_acc: 0.8050
Epoch 6/30
50/50 [==============================] - 81s 2s/step - loss: 0.3157 - acc: 0.8790 - val_loss: 0.4845 - val_acc: 0.8200
Epoch 7/30
50/50 [==============================] - 80s 2s/step - loss: 0.2493 - acc: 0.9020 - val_loss: 0.4803 - val_acc: 0.8400
Epoch 8/30
50/50 [==============================] - 80s 2s/step - loss: 0.2765 - acc: 0.8

The model shows a lot of improvement after finetuning with validation accuracy reaching 88%. This is a sufficient demonstration of how powerful a tool transfer learning is for machine leanring in data-poor fields such as biomedical imaging etc.